In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
PROJECT_PREFIX = os.getenv("PROJECT_PREFIX")
PROJECT_PREFIX_UNDERSCORE = os.getenv("PROJECT_PREFIX_UNDERSCORE")
DATA_LAKE_BUCKET = os.getenv("DATA_LAKE_BUCKET")
ICEBERG_LOCK_TABLE = os.getenv("ICEBERG_LOCK_TABLE")
TEST_DB = f"{PROJECT_PREFIX_UNDERSCORE}_iceberg_test_db"

In [3]:
spark = (
    SparkSession.builder
    .appName("GlueIcebergLocalTest")
    .master("local[*]")
    .config(
        "spark.jars.packages",
        "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1,"
        "org.apache.iceberg:iceberg-aws-bundle:1.6.1,"
        "org.apache.hadoop:hadoop-aws:3.3.4"
    )
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.fast.upload", "true")
    .config("spark.hadoop.fs.s3a.multipart.size", "104857600")  # 100MB
    .config("spark.hadoop.fs.s3a.connection.maximum", "1000")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.glue_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.glue_catalog.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
    .config("spark.sql.catalog.glue_catalog.warehouse", f"s3://{DATA_LAKE_BUCKET}/")
    .config("spark.sql.catalog.glue_catalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.glue_catalog.lock.table", f"{ICEBERG_LOCK_TABLE}")
    .config("spark.sql.defaultCatalog", "glue_catalog")
    .getOrCreate()
)

25/11/19 15:48:07 WARN Utils: Your hostname, Nguyens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
25/11/19 15:48:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/anhtu/.ivy2/cache
The jars for the packages stored in: /Users/anhtu/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-54e1b017-880d-411d-a39e-7bfd086c483c;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/anhtu/.pyenv/versions/3.11.11/envs/crypto-cloud/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.6.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 111ms :: artifacts dl 4ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.apache.iceberg#iceberg-aws-bundle;1.6.1 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-------------------------------

In [4]:
spark.sql(f"""
CREATE DATABASE IF NOT EXISTS {TEST_DB}
LOCATION 's3://{DATA_LAKE_BUCKET}/iceberg_test_db/'
""")

DataFrame[]

In [5]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {TEST_DB}.sample_orders (
    order_id INT,
    product STRING,
    price DOUBLE
) USING iceberg
TBLPROPERTIES ('format-version'='2')
""")

DataFrame[]

In [6]:
spark.sql(f"""
INSERT INTO {TEST_DB}.sample_orders
VALUES (3, 'BTC', 62500.1), (4, 'ETH', 3100.5)
""")

DataFrame[]

In [ ]:
spark.sql(f"SELECT * FROM {TEST_DB}.sample_orders").show()

+--------+-------+-------+
|order_id|product|  price|
+--------+-------+-------+
|       3|    BTC|62500.1|
|       4|    ETH| 3100.5|
|       3|    BTC|62500.1|
|       4|    ETH| 3100.5|
+--------+-------+-------+



25/11/19 15:50:45 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /private/var/folders/21/8r4f38z90tz5zgftyt7rck640000gn/T/blockmgr-7e84e2e8-737f-4eec-924d-73c7eef241f9. Falling back to Java IO way
java.io.IOException: Failed to delete: /private/var/folders/21/8r4f38z90tz5zgftyt7rck640000gn/T/blockmgr-7e84e2e8-737f-4eec-924d-73c7eef241f9
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:174)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:368)
	at org.apache.spark.storage.DiskBl